In [ ]:
# Dash with Datatable, Dropdowns and Charts

import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output
from dash import dash_table, dcc, html
import dash_bootstrap_components as dbc

# Import for stand-alone program
# from dash import Dash

# Import for Jupyter
from jupyter_dash import JupyterDash # pip install jupyter-dash

app = JupyterDash(__name__, prevent_initial_callbacks=True,
                  external_stylesheets=[dbc.themes.BOOTSTRAP], 
                  meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1.0'}]
                 )

# Declare server for Heroku deployment. Needed for Procfile.
# server = app.server

df = pd.DataFrame(
    {
        "patient_id": ["0001", "0002", "0003", "0004"],
        "first_name": ["Alice", "Bob", "Carol", "David"],
        "dob": ["1950-01-01", "1950-02-02", "1950-03-03", "1950-04-04"],
        "value1": [20, 30, 40, 50],
        "value2": [10, 2, 8, 4],
    }
)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H4("Dash with Datatable, Dropdowns and Charts",
                    className='text-center text-default'),
            html.Label("A Dash datatable connected with dropdowns and plotly charts. Editing the datatable will update the charts. Use the dropdown menus to select which data to display in the charts.",
                    className='text-center text-default mb-4'),
            dash_table.DataTable(
                id='datatable_id',
                data=df.to_dict('records'),
                columns=[
                    {"name": i, "id": i, "deletable": True, "selectable": True, "hideable": True} for i in df.columns
                ],
                editable=True,
                filter_action="native",
                sort_action="native",
                sort_mode="multi",
                row_selectable="multi",
                row_deletable=True,
                selected_columns=[],
                selected_rows=[],
                page_action="native",
                page_current= 0,
                page_size= 6,
            ),
            html.Hr(),
        ], width={'size':12, 'offset':0}),
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("Select either Value 1 or Value 2 for the bar chart"),
            dcc.Dropdown(id='bar_dropdown',
                options=[
                    {'label': 'Value 1', 'value': 'value1'},
                    {'label': 'Value 2', 'value': 'value2'}
                ],
                value='value1',
                multi=False,
                clearable=False
            ),
            dcc.Graph(id='barchart_container'),
        ], # width={'size':6, 'offset':0, 'order':1}
        xs=12, sm=12, md=6, lg=6, xl=6),
        dbc.Col([
            html.Label("Select either Value 1 or Value 2 for the pie chart"),
            dcc.Dropdown(id='pie_dropdown',
                options=[
                    {'label': 'Value 1', 'value': 'value1'},
                    {'label': 'Value 2', 'value': 'value2'}
                ],
                value='value2',
                multi=False,
                clearable=False
            ),
            dcc.Graph(id='piechart'),
        ], # width={'size':6, 'offset':0, 'order':2}
        xs=12, sm=12, md=6, lg=6, xl=6),
    ]),
], fluid=True) # Remove container spacing

@app.callback(
    [Output('barchart_container', 'figure'),
     Output('piechart', 'figure')],
    [Input(component_id='datatable_id', component_property="derived_virtual_data"),
     Input(component_id='datatable_id', component_property='derived_virtual_selected_rows'),
     Input(component_id='datatable_id', component_property='derived_virtual_selected_row_ids'),
     Input(component_id='datatable_id', component_property='selected_rows'),
     Input(component_id='datatable_id', component_property='derived_virtual_indices'),
     Input(component_id='datatable_id', component_property='derived_virtual_row_ids'),
     Input(component_id='datatable_id', component_property='active_cell'),
     Input(component_id='datatable_id', component_property='selected_cells'),
     Input('bar_dropdown', 'value'),
     Input('pie_dropdown', 'value')]
)
def update_bar(all_rows_data, slctd_row_indices, slct_rows_names, slctd_rows,
               order_of_rows_indices, order_of_rows_names, actv_cell, slctd_cell,
               bar_dropval, pie_dropdown):

    dff = pd.DataFrame(all_rows_data)

    if "first_name" in dff and "value1" in dff:
        barchart_figure = px.bar(
            data_frame=dff,
            x="first_name",
            y=bar_dropval,
            labels={"first_name": "First Name", "value1": "Value 1"}
        )
        pie_chart = px.pie(
            data_frame=dff,
            names='first_name',
            values=pie_dropdown,
            hole=.3,
            labels={'first_name': 'Names'}
        )
        return barchart_figure, pie_chart


"""
if __name__ == '__main__':
    app.run_server(debug=True)
"""

# Run app and display result inline in the notebook
app.run_server(port=8050, mode='inline')
